In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
import math
from scipy.stats import chi2
from scipy import stats

In [ ]:
# functions

# check whether the taxa is in the pathway abundance dataset.
def check(taxa_name1, taxa_name2, pw_data):
    """
    taxa_name1: str, genus name of taxon
    taxa_name2: str, species name of taxon
    pw_data: pandas.dataframe, pathway abundance matrix
    """
    if len(pw_data[(pw_data['pw_taxa'].str.contains(taxa_name1)) & (pw_data['pw_taxa'].str.contains(taxa_name2))]['pw_taxa']) > 0:
        return True
    return False

# take the samples with targeted disease
def take_sample(data):
    """
    data: pandas.dataframe, the metadata.
    """
    data.loc[data['disease'].str.contains('T2D'), 'disease'] = 'T2D'
    data.loc[data['disease'].str.contains('healthy'), 'disease'] = 'healthy'
    data = data[data['disease'].isin(['T2D', 'healthy'])].reset_index().drop(['index'], axis = 1)
    return data

# grabbing all the pathways associated with the query taxa from pathway abundance dataset
def grab_pw_data(rawdata, metadata, pathway_data, phylum_name, species_name):
    """
    rawdata: pandas.dataframe, metagenomice counts dataset.
    metadata: pandas.dataframe, metadata.
    pathway_data: pandas.dataframe, pathway abundance dataset.
    phylum_name: str, genus name of taxon
    species_name: str, species name of taxon
    """
    if check(phylum_name, species_name, pathway_data):   # Check whether a taxon is in pathway abundance dataset.
        all_data = label_data(rawdata, metadata)
        all_data = all_data[all_data['disease'].isna() == False].reset_index(drop = True)
        samples_used = take_sample(all_data)[['sample_id', 'disease']]

        pw_extract = pathway_data[pathway_data['pw_taxa'].str.contains(phylum_name) & pathway_data['pw_taxa'].str.contains(species_name)].reset_index().drop(['index'], axis = 1)
        cols = pw_extract.pw_taxa
        pw_extract_t = pw_extract.T; pw_extract_t.columns = cols
        pw_extract_t = pw_extract_t.drop(['pw_taxa', 'FeatureID']).reset_index().rename(columns = {'index':'sample_id'})
        return pw_extract_t.merge(samples_used)
    
    return False

In [ ]:
# import taxa list derived from OS scenario
# ld_train = pd.read_csv(r"C:\Users\edwar\Desktop\Melbourne\research_project\loadings_training.csv")
# ld_prjeb = pd.read_csv(r"C:\Users\edwar\Desktop\Melbourne\research_project\loadings_overlappedprjeb.csv")
ld_qinj = pd.read_csv(r"C:\Users\edwar\Desktop\Melbourne\research_project\loadings_overlappedqinj.csv")

# filter the samples undergoing treatments
qin_metadata = qin_metadata[qin_metadata['treatment'] == 'no'].reset_index(drop = True)

Finding the taxa which exists in pathway abundance dataset

In [ ]:
# Explore taxa-pathway
taxa_lst = [i for i in ld_qinj['Unnamed: 0']]
perc_thres = 0.4
pw_df = pd.DataFrame()
for taxa in taxa_lst:
    # revise specific taxa name
    if 'phocaeicola' in taxa:
        taxa = taxa.replace('phocaeicola', 'bacteroides')

    if len(taxa.split('.')) == 2 or len(taxa.split('.')) == 3:
        phy_name, spe_name = taxa.split('.')[0].capitalize(), taxa.split('.')[1]; #print(phy_name, spe_name)
    elif (len(taxa.split('.')) == 4) and ('cag' in taxa.split('.')):
        phy_name, spe_name = taxa.split('.')[0].capitalize(), taxa.split('.')[1]; #print(phy_name, spe_name)
    elif len(taxa.split('.')) == 4:
        phy_name, spe_name = taxa.split('.')[1].capitalize(), taxa.split('.')[3]; #print(phy_name, spe_name)
        
    df = grab_pw_data(rawdata = qin_rawdata, metadata = qin_metadata, phylum_name = phy_name, species_name = spe_name, pathway_data = qin_pw)
    if type(df) == bool:
        continue
    print(taxa)
    
    df1 = df.iloc[:, 3:]; df1['sample_id'] = df['sample_id']
    pw_lst = list(df1.columns)
    # record derived pathways in this taxa

    # Count the non-zero percentage of the specific pathways
    for IDX in range(0, len(pw_lst) - 2):
        df_temp = df1[[pw_lst[IDX], pw_lst[-2]]]
        perc_t2d = (df_temp[df_temp['disease'] == 'T2D'].iloc[:, 0] != 0).sum() / len(df_temp[df_temp['disease'] == 'T2D'].iloc[:, 0])
        perc_hea = (df_temp[df_temp['disease'] == 'healthy'].iloc[:, 0] != 0).sum() / len(df_temp[df_temp['disease'] == 'healthy'].iloc[:, 0])
        
        if perc_t2d >= perc_thres or perc_hea >= perc_thres:
            df_temp = pd.DataFrame()
            df_temp['taxa'] = [phy_name + ' ' + spe_name]
            df_temp['pathway'] = [pw_lst[IDX].split('|')[0]]
            df_temp['taxa-pathway'] = [pw_lst[IDX]]
            df_temp['T2D%'] = [perc_t2d]
            df_temp['health%'] = [perc_hea]
            df_temp['diff%'] = [abs(perc_t2d - perc_hea)]
            if pw_df.empty:
                pw_df = df_temp
            else:
                pw_df = pd.concat([pw_df, df_temp], ignore_index = True)

pw_df = pw_df.sort_values(by = ['diff%'], ascending = False, ignore_index = True)

Permutation stats test

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, balanced_accuracy_score

covdata = qin_cov; metadata = qin_metadata; n_perm = 1000
uni_pathway = list(pw_df['pathway'].unique())
pval_pathway = []

testdf = pd.DataFrame()
testdf['pathway'] = uni_pathway
testdf['p-value'] = ['' for i in range(len(uni_pathway))]

ROUND_RECORD = 0
for pathway in uni_pathway:
# pathway = uni_pathway[0]
    taxa_pws = list(pw_df[pw_df['pathway'] == pathway].reset_index(drop = True)['taxa-pathway'])
    X_t = covdata[covdata['pw_taxa'].isin(taxa_pws)].reset_index(drop = True)
    X = X_t.T.drop(['pw_taxa', 'FeatureID'])
    X.columns = list(X_t['pw_taxa'])
    X = X.reset_index().rename(columns = {'index':'sample_id'}).merge(metadata[['subject_id', 'disease']], left_on = ['sample_id'], right_on = ['subject_id']).drop(['subject_id'], axis = 1)

    # start permutation
    model = LogisticRegression().fit(X = X.iloc[:, 1:-1].astype(float), y = X.iloc[:, -1].astype('category').cat.codes)
    obs_acc = balanced_accuracy_score(y_true = X.iloc[:, -1].astype('category').cat.codes, y_pred = model.predict(X.iloc[:, 1:-1].astype(float)))
    counts = 0
    for i in range(n_perm):
        perm_acc = balanced_accuracy_score(y_true = X.iloc[:, -1].sample(n = X.shape[0], replace = False).reset_index(drop = True).astype('category').cat.codes, 
                                  y_pred = model.predict(X.iloc[:, 1:-1].astype(float)))
        if perm_acc > obs_acc:
            counts += 1
    pval_pathway.append(counts/n_perm)
    ROUND_RECORD += 1
    if (ROUND_RECORD + 1) % 10 == 0:
        print(ROUND_RECORD + 1)

testdf['p-value'] = pval_pathway

In [ ]:
testdf1 = testdf.merge(pw_df['pathway'].value_counts().reset_index(), on = ['pathway'])
p_thres = 0.005
testdf1 = testdf1[testdf1['p-value'] <= p_thres].reset_index(drop = True)
ttl_ntaxa = find_taxaNum(qin_pw) # no.of taxa within background
ttl_sampled_taxa = pw_df['taxa'].nunique()  # no.of taxa in our sampled list  pw_df['taxa'].unique()
testdf1 = testdf1.merge(all_pwtaxa_counts, how = 'left', on = ['pathway'])
testdf1 = testdf1.rename(columns = {'count':'no_taxafound'})

Hypergeometric test for pathway enrichment

In [ ]:
from scipy.stats import hypergeom
k = testdf1['no_taxafound']  # k: no.of taxa found in pathway by us (taxa out of ttl_sampled_taxa).
hypergeom_p = hypergeom.sf(k - 1, M = ttl_ntaxa, n = testdf1['taxa_counts'], N = ttl_sampled_taxa)
testdf1['hypergeom_p'] = hypergeom_p
testdf2 = testdf1[testdf1['hypergeom_p'] <= 0.05].reset_index(drop = True)
testdf2

Visualization

In [ ]:
testdf2['-log10p'] = -np.log10(testdf2['hypergeom_p'])
testdf2 = testdf2.sort_values(by = ['-log10p'], ascending = True, ignore_index = True)

plt.rcParams['font.family'] = 'Arial'
plt.figure(figsize=(8, 10))
scatter = plt.scatter(
    testdf2['-log10p'],
    testdf2['pathway'],
    s = testdf2['taxa_counts'],    # size (you can adjust the multiplier to make it larger)
    c = testdf2['no_taxafound'],        # color
    cmap = 'RdYlGn_r',             # Red -> Yellow -> Green reversed
    alpha = 0.7,
    edgecolors='black'
)
plt.axvline(x=-np.log10(0.05), color = 'red', linestyle = '-', linewidth = 0.5)

# set background color
ax = plt.gca()
ax.set_facecolor('#F5F5DC')
ax.set_xlim(left=1.0, right=3)

# Colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('No. of taxa identified by model in pathway abundance dataset', fontsize=13, loc = 'center')

# Labels
plt.xlabel(r'$-\log_{10}$(hypergeometric p-value)', fontsize=13)
plt.ylabel('Pathway', fontsize=13)
plt.title('Pathway Enrichment', fontsize=15)
plt.xticks(fontsize = 11.5)

plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout(rect=[0, 0, 0.95, 1])
handles, _ = scatter.legend_elements(prop='sizes', alpha=0.7)
labels = ['50', '200', '400']
plt.legend(handles, labels, title = 'no.of taxa in pathways')
plt.show()